In [1]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
import os

groq_api_key = os.environ["GROQ_API_KEY"]

In [ ]:
import yfinance as yf
yf.__version__


'0.2.62'

In [7]:
from langchain_groq import ChatGroq

llama3 = ChatGroq(api_key=groq_api_key, model="llama3-8b-8192")

llama3

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E44C556900>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E44C557620>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
llama3.invoke("Hello, How are you?")

AIMessage(content='I\'m just a language model, I don\'t have feelings or emotions like humans do, so I don\'t have a personal experience of being "good" or "bad" or feeling any particular way. I exist solely to provide information and assist with tasks to the best of my ability! I\'m here to help you with any questions or topics you\'d like to discuss, though! How about you? How\'s your day going?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 16, 'total_tokens': 104, 'completion_time': 0.073333333, 'prompt_time': 0.002464924, 'queue_time': 0.167290951, 'total_time': 0.075798257}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--0f761109-f072-4513-b410-91c689c9e44d-0', usage_metadata={'input_tokens': 16, 'output_tokens': 88, 'total_tokens': 104})

 Define Tools

In [9]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""
    
    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()
    
    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)   
    inst_holders = ticker_obj.get_institutional_holders()
    
    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)
    
    curr_year = date.today().year
    
    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]
    
    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()
    
    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_news()

 Define Agent (LLM + Tools)

In [11]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

tools = [
         company_information,
         last_dividend_and_earnings_date,
         stock_splits_history,
         summary_of_mutual_fund_holders,
         summary_of_institutional_holders, 
         stock_grade_updrages_downgrades,
         stock_news
]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Try to answer user query using available tools.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llama3 = ChatGroq(api_key=groq_api_key, model="llama3-8b-8192", temperature=0)

finance_agent = create_tool_calling_agent(llama3, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [12]:
from langchain_core.messages import HumanMessage

resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is address of microsoft?")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'MSFT'}`


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, includ

In [13]:
print(resp["output"])

The address of Microsoft is One Microsoft Way, Redmond, WA 98052-6399, United States.


In [14]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is last close price of microsoft?")]})
print(resp["output"])



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'MSFT'}`


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, includ

In [17]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is dividend yield for microsoft stock?")]})
print(resp["output"])



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'MSFT'}`


{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, includ

In [18]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'NVDA'}`


{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation, a computing infrastructure company, provides graphics and compute and networking solutions in the United States, Singapore, Taiwan, China, Hong Kong, and internationally. The Compute & Networking segment comprises Data Center computing platforms and end-to-end networking platforms, including Quantum for InfiniBand and Spectrum for Ethernet; NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other embedded platforms; NVIDIA AI Enterprise and 

In [19]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Summarize Key people at apple")]})



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'AAPL'}`


{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover 

In [20]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Top institutional holders of microsoft stock.")]})
print(resp["output"])



> Entering new AgentExecutor chain...

Invoking: `summary_of_institutional_holders` with `{'ticker': 'MSFT'}`


[{'Date Reported': Timestamp('2025-03-31 00:00:00'), 'Holder': 'Vanguard Group Inc', 'pctHeld': 0.093, 'Shares': 691386214, 'Value': 324927319530, 'pctChange': 0.0107}, {'Date Reported': Timestamp('2025-03-31 00:00:00'), 'Holder': 'Blackrock Inc.', 'pctHeld': 0.0785, 'Shares': 583284931, 'Value': 274123500461, 'pctChange': 0.012200001}, {'Date Reported': Timestamp('2025-03-31 00:00:00'), 'Holder': 'State Street Corporation', 'pctHeld': 0.0398, 'Shares': 296030244, 'Value': 139123852537, 'pctChange': 0.0031}, {'Date Reported': Timestamp('2025-03-31 00:00:00'), 'Holder': 'FMR, LLC', 'pctHeld': 0.026199998, 'Shares': 194591253, 'Value': 91451077503, 'pctChange': -0.0148}, {'Date Reported': Timestamp('2025-03-31 00:00:00'), 'Holder': 'Geode Capital Management, LLC', 'pctHeld': 0.023599999, 'Shares': 175469697, 'Value': 82464615508, 'pctChange': 0.0437}, {'Date Reported': Timest

In [21]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Which firms recently changed grade of apple stock? Also, return rating changes.")]})
print(resp["output"])



> Entering new AgentExecutor chain...

Invoking: `stock_grade_updrages_downgrades` with `{'ticker': 'AAPL'}`


[{'Firm': 'Needham', 'ToGrade': 'Hold', 'FromGrade': 'Buy', 'Action': 'down', 'priceTargetAction': 'Maintains', 'currentPriceTarget': 225.0}, {'Firm': 'Rosenblatt', 'ToGrade': 'Neutral', 'FromGrade': 'Buy', 'Action': 'down', 'priceTargetAction': 'Lowers', 'currentPriceTarget': 217.0}, {'Firm': 'Jefferies', 'ToGrade': 'Underperform', 'FromGrade': 'Hold', 'Action': 'down', 'priceTargetAction': 'Raises', 'currentPriceTarget': 170.62}, {'Firm': 'Keybanc', 'ToGrade': 'Sector Weight', 'FromGrade': 'Underweight', 'Action': 'up', 'priceTargetAction': '', 'currentPriceTarget': 0.0}, {'Firm': 'Jefferies', 'ToGrade': 'Hold', 'FromGrade': 'Underperform', 'Action': 'up', 'priceTargetAction': 'Lowers', 'currentPriceTarget': 167.88}, {'Firm': 'Oppenheimer', 'ToGrade': 'Perform', 'FromGrade': 'Outperform', 'Action': 'down', 'priceTargetAction': '', 'currentPriceTarget': 0.0}, {'Firm': 'Pipe